In [1]:
import utils.extract_df as extract_df
import utils.transform as transform 
import utils.main as main
import streamlit as st
import pandas as pd

In [2]:
extract_df.readcsv('data/taxi_data.csv').shape

(90000, 19)

In [3]:
extract_df.readshp('data/nyc-boundaries/geo_export_9ca5396d-336c-47af-9742-ab30cd995e41.shp')

,boro_code,boro_name,shape_area,shape_leng,geometry
0,5.0,Staten Island,1.623621e+09,325917.353950,"MULTIPOLYGON (((-74.05051 40.56642, -74.05047 ..."
1,1.0,Manhattan,6.365205e+08,357713.308162,"MULTIPOLYGON (((-74.01093 40.68449, -74.01193 ..."
2,2.0,Bronx,1.187175e+09,463180.579449,"MULTIPOLYGON (((-73.89681 40.79581, -73.89694 ..."
3,3.0,Brooklyn,1.934138e+09,728146.574928,"MULTIPOLYGON (((-73.86327 40.58388, -73.86381 ..."
4,4.0,Queens,3.041419e+09,888199.731385,"MULTIPOLYGON (((-73.82645 40.59053, -73.82642 ..."


In [4]:
df_transform = transform.dataTransformation(df = extract_df.readcsv('data/taxi_data.csv'), 
nyc = extract_df.readshp('data/nyc-boundaries/geo_export_9ca5396d-336c-47af-9742-ab30cd995e41.shp'))

In [5]:
df_transform = df_transform.transform()

In [6]:
(pd.to_datetime(df_transform.pickup_datetime.min()), pd.to_datetime(df_transform.pickup_datetime.max()))

(Timestamp('2014-01-01 00:24:00+0000', tz='UTC'),
 Timestamp('2015-06-30 23:57:04+0000', tz='UTC'))

In [7]:
df_transform.columns

Index(['dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'feat01',
       'feat02', 'feat03', 'feat04', 'feat05', 'feat06', 'feat07', 'feat08',
       'feat09', 'feat10', 'passenger_count', 'pickup_datetime',
       'pickup_latitude', 'pickup_longitude', 'passenger_big_group',
       'fare_amount_log', 'year', 'month', 'day', 'hour', 'trip_distance'],
      dtype='object')

## Run the application

In [8]:
! streamlit run Homepage.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.1.2.172:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
/Users/irenaz/.pyenv/versions/3.9.18/lib/python3.9/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
^C
  Stopping...
